In [1]:
import torch 
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import boto3
import pickle
from sklearn.metrics import roc_auc_score
import matplotlib
import matplotlib.pyplot as plt
import sys
from torchvision import transforms
import math
import torch.optim as optim
import time
import json

**Verify that GPUs are availble, will use all**

In [2]:
print(torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

8
cuda


In [3]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, num_classes, num_events=500, seq_length=120, dropout=0.5):
        '''
        Initialize a transformer model for adverse events. The model consists of the following:
        - Transformer encoder layers
        - Single 1D CNN layer
        - Final fully connected layer to determine probability of readmissions
        
        Args:
            
            ntoken: number of tokens in embedding layer (vocabulary size)
            ninp: embedding dimension (number of inputs)
            
            nhead: number of heads in transformers
            nhid: number of transformer linear dimensions
            
            nlayers: number of layers in transfromer
            
            num_classes: number of classes to predict (in this case, binary)
            
            seq_length: dimension of linear layer output
            num_events: maximum number of events per patient
            
            dropout: strength of regularization
        '''
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        print("parameters: embsize:{}, nhead:{}, nhid:{}, nlayers:{}, dropout:{}".format(ninp, nhead, nhid, nlayers, dropout))
        
        # Inputs into transformer: mask for padding and embeddings
        self.src_mask = None
        self.event_emb = nn.Embedding(ntoken, ninp)
        
        # Transformer layer
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        
         # CNN & fully connected layers
        
        self.ff = nn.Linear(int(num_events), int(seq_length))
        self.fc = nn.Linear(int(seq_length), num_classes)
        self.nonlinear = nn.ReLU()
        self.softmax = nn.Softmax(-1)
        self.Conv1d = nn.Conv1d(ninp, 1, 1, stride=1)
        
        # record
        self.ninp = ninp
        self.dropout = dropout
        self.num_events= num_events
        self.num_classes = num_classes
        
        # initalize weights
        self.init_weights()
    

    def init_weights(self):
        '''Initialize weights in embedding and fully connected layers'''
        initrange = 0.1
        self.event_emb.weight.data.uniform_(-initrange, initrange)
        
        self.fc.bias.data.zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)
        
        self.ff.bias.data.zero_()
        self.ff.weight.data.uniform_(-initrange, initrange)

        
    def forward(self, src, mask=None, pos=None): 
        '''
        Forward propagation steps:
        - convert events into embedding vectors & positional encoding
        - transformer encoder layers
        - CNN layer
        - final 
        Notes:
        no position encoding here, no obvious clues for sequential or order found 
        '''        
        if mask is not None:
            #src_key_padding_mask needs boolean mask
            src_mask = (mask == 0)

        src = self.event_emb(src).transpose(0,1) * math.sqrt(self.ninp)
        
        trans_output = self.transformer_encoder(src, src_key_padding_mask=src_mask).transpose(0, 1).transpose(1,2)
        final_feature_map = self.Conv1d(trans_output).squeeze()
        
        out_mask = mask.float().masked_fill(mask == 0.0, float(-100.0)).masked_fill(mask == 1.0, float(0.0)).view(mask.size()[0], -1)
        # extract normalized feature importances per prediction
        importance_out = self.softmax(final_feature_map+out_mask)
        
        output = self.ff(final_feature_map)
        output = self.nonlinear(output)
        output = self.fc(output)

        return output, importance_out

In [4]:
'''
no position encoding here, but save it for the future in case
'''
from torch.autograd import Variable
class PositionalEncoding(nn.Module):

    def __init__(self, events, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)
        #self.pe = torch.zeros(max_len, d_model)
        #self.pe = torch.zeros(pos.size()[0], , self.d_model)
        ''' 
        #position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        position = torch.tensor([float(i) for i in range(dim)] * seq_length + [0.0] * (max_len - seq_length * dim), dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)'''

    def forward(self, x, pos):

        #position1 = torch.tensor([float(i) for i in range(30)] * 30 + [0.0] * (5000 - 30 * 30), dtype=torch.float).unsqueeze(1)
        pe = torch.zeros(x.size()).cuda()
        pe = pe.transpose(0, 1)
        position = pos.unsqueeze(2)
        div_term = torch.exp(torch.arange(0, self.d_model, 2).float() * (-math.log(10000.0) / self.d_model)).cuda()
        pe[:, :, 0::2] = torch.sin(position * div_term).cuda()
        pe[:, :, 1::2] = torch.cos(position * div_term).cuda()
        pe = pe.transpose(0, 1)
        #print(position1.size(), position.size(), div_term.size())
        #self.pe[:, 0::2] = torch.sin(position * div_term)
        #self.pe[:, 1::2] = torch.cos(position * div_term)
        #self.pe = pe.unsqueeze(0).transpose(0, 1)
        #self.register_buffer('pe', pe)
        x = x + pe[:x.size(0), :]
        return self.dropout(x)
    

## Dataset and data loader functions to create batches

In [5]:
class BuildDataset(Dataset):
    '''
    Read in dataset, if data is already split into train, test, and/or validation sets.
    
    ProcessData: extract input, labels, mask from an existing Python object (via pickle or otherwise)
    '''
    def __init__(self, data_file, event_length=500, data_list=None, mode='read'): 
        if mode != 'read' and data_list != None:
            self.data, self.label, self.mask = self.ProcessData(data_list, event_length)
            
    def ProcessData(self, data_list, event_length):
        input_data, labels, mask = data_list[0][:,-event_length:], data_list[1], data_list[2][:,-event_length:]
        return input_data, labels, mask
    
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return torch.tensor(self.data[idx]), torch.tensor(self.label[idx]), torch.tensor(self.mask[idx])

In [6]:
# Read data from the saved pickle file, the order is [patietn_id, input_data, labels, mask_for_padding, position(no used here)]
def ReadData(file_dir, event_length):
    with open(file_dir, 'rb') as f:
        ids, data, label, mask = pickle.load(f)
        ids = ids.astype(str)
        cut_data = data[:,-event_length:]
        cut_mask = mask[:,-event_length:]
        #cut_pos = pos[:,-event_length:]
        label = label.astype(int)
    return ids, cut_data, label, cut_mask#, cut_pos

## Training and Evaluation functions

In [7]:
def EopochTrain(model, dataloader, optimizer, criterion, device="cuda", metric='acc'):
    '''
    Model training, called by ModelProcess function
    
    Note: Does not return prediction importance scores
    '''
    epoch_loss = 0
    epoch_metric = 0
    
    model.train()
    
    order_labels = None
    prediction_scores = None
    for idx, [seq, labels, mask] in enumerate(dataloader):

        optimizer.zero_grad()
        
        labels = labels.squeeze().float()
        seq, labels, mask = seq.cuda(), labels.cuda(), mask.cuda()
        
        predictions, _ = model(seq, mask=mask)
        
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        if order_labels is None:
            order_labels = labels.cpu().numpy()
            prediction_scores = torch.sigmoid(predictions).detach().cpu().numpy()
        else:
            order_labels = np.concatenate((order_labels, labels.cpu().numpy()))
            prediction_scores =np.concatenate((prediction_scores, torch.sigmoid(predictions).detach().cpu().numpy()))
    
        epoch_loss += loss.item()
        
    if metric == 'acc':
        epoch_metric = get_average_accuracy(prediction_scores, order_labels)
    elif metric == 'auc':
        epoch_metric = roc_auc_score(order_labels, prediction_scores, average='weighted')
    return epoch_loss / len(dataloader), epoch_metric

In [18]:
def EopochVal(model, dataloader, optimizer, criterion, device="cuda", metric='auc'):
    '''
    Evaluate model performance, called by ModelProcess function
    
    Returns predictions, metrics and importance scores
    '''
    epoch_loss = 0
    epoch_metric = 0
    
    model.eval()
    
    order_labels = None
    prediction_scores = None
    events = None
    important_scores = None
    
    for idx, [seq, labels, mask] in enumerate(dataloader):
        labels = labels.squeeze().float()
        seq, labels, mask = seq.cuda(), labels.cuda(), mask.cuda()
        predictions, importance = model(seq, mask=mask)

        loss = criterion(predictions, labels)
        
        if order_labels is None:
            order_labels = labels.cpu().numpy()
            prediction_scores = torch.sigmoid(predictions).detach().cpu().numpy()
            events = seq.cpu().numpy()
            important_scores = importance.detach().cpu().numpy()
        else:
            order_labels = np.concatenate((order_labels, labels.cpu().numpy()))
            prediction_scores =np.concatenate((prediction_scores, torch.sigmoid(predictions).detach().cpu().numpy()))
            events = np.concatenate((events, seq.cpu().numpy()))
            important_scores = np.concatenate((important_scores, importance.detach().cpu().numpy()))
            
        epoch_loss += loss.item()
        
    if metric == 'acc':
        epoch_metric = get_average_accuracy(prediction_scores, order_labels)
    elif metric == 'auc':
        epoch_metric = roc_auc_score(order_labels, prediction_scores)
        
    return epoch_loss / len(dataloader), epoch_metric, [order_labels, events, important_scores, prediction_scores]

In [21]:
def TrainingProcess(model, epoch, criterion, dataloaders:list, month):

    if torch.cuda.is_available():
        device="cuda"
    else:
        device="cpu"
    
    print("device: ", device)
    
    optimizer = optim.AdamW(model.parameters())
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

    pre_val_metric = 0.0
    final_test_metric = 0.0
    train_loss=[]
    train_metrics=[]
    #val_loss=[]
    #test_loss=[]

    for i in range(epoch):
        print('-' * 10)
        print('Epoch {}/{}'.format(i+1, epoch))
        print('-' * 10)
        epoch_train_loss, epoch_train_metric = EopochTrain(model, dataloaders[0], optimizer, criterion, device="cuda", metric='auc')

        train_metrics.append(epoch_train_metric)
        print('epoch_train_loss:',np.mean(epoch_train_loss), 'epoch_train_metric:', np.mean(epoch_train_metric))
        
        epoch_val_loss, epoch_val_metric, val_results = EopochVal(model, dataloaders[1], optimizer, criterion, device="cuda", metric='auc')
        #val_loss.append(epoch_val_loss)
        print('epoch_val_loss:',np.mean(epoch_val_loss), 'epoch_val_metric:', np.mean(epoch_val_metric))
        
        epoch_test_loss, epoch_test_metric, importance_results = EopochVal(model, dataloaders[-1], optimizer, criterion, device="cuda", metric='auc')
        #test_loss.append(epoch_test_loss)
        print('epoch_test_loss:',np.mean(epoch_test_loss), 'epoch_test_metric:', np.mean(epoch_test_metric))
        
        #save model based on val results
        #epoch_val_metric = epoch_train_metric
        if epoch_val_metric > pre_val_metric:
            
            print("updated")
            pre_val_metric = epoch_val_metric
            final_test_metric = epoch_test_metric
            final_importance_results = importance_results
            
            torch.save(model.module.state_dict(), './vocab120_model_weights_month-{}_emsize-{}_head-{}_layers-{}_valauc-{}.pth'.format(month, emsize, nhead, nlayers, np.round(epoch_val_metric, decimals=3)))
        #scheduler.step()
        
    return train_metrics, pre_val_metric, final_test_metric, final_importance_results

In [11]:
#Load vocab
vocab = torch.load('data/ae_pos_vocab_last120_whole_non3')
vocab.itos[52]


'h_99284'

## Model training & evaluation

In [22]:


train_batch_size = 10000
test_batch_size = 10000

ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 16 # embedding dimension
nhid = 32 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 1 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
n_class = 20

test_metrics = []
results = []


#criterion to use
criterion = nn.BCEWithLogitsLoss(pos_weight=None).cuda()
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, n_class, num_events=500, seq_length=120, dropout=0.5).to(device)

if torch.cuda.is_available():
    model = model.cuda()
if torch.cuda.device_count()>1:
    model = nn.DataParallel(model)
    
# Training month by month
for i in range(1, 12):
    if i < 10:
        m = '0' + str(i)
    else:
        m = str(i)
    whole_ids, whole_data, whole_labels, whole_mask = ReadData('data/ae_process/np_ae_last120_vocab120_non3digit_month' + m + '.pkl', event_length=500)
    
    train_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    val_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    test_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    
    train_index = np.where(np.in1d(whole_ids, train_ids))[0]
    val_index = np.where(np.in1d(whole_ids, val_ids))[0]
    test_index = np.where(np.in1d(whole_ids, test_ids))[0]
    print('whole data is done')
    

    X_train, y_train, mask_train  = whole_data[train_index], whole_labels[train_index], whole_mask[train_index]
    X_val, y_val, mask_val = whole_data[val_index], whole_labels[val_index], whole_mask[val_index]
    X_test, y_test, mask_test = whole_data[test_index], whole_labels[test_index], whole_mask[test_index]

    train_dataset = BuildDataset('', event_length=500, data_list=[X_train, y_train, mask_train], mode='load')
    val_dataset = BuildDataset('', event_length=500, data_list=[X_val, y_val, mask_val], mode='load')
    test_dataset = BuildDataset('', event_length=500, data_list=[X_test, y_test, mask_test], mode='load')

    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=6, drop_last=True)
    val_dataloader = DataLoader(val_dataset, batch_size=train_batch_size, shuffle=False, num_workers=6, drop_last=True)
    test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=6, drop_last=True)



    epoch=2
    train_metrics, val_metric, test_metric, final_importance_results = TrainingProcess(model, epoch, criterion, [train_dataloader, val_dataloader, test_dataloader], m)

    final_importance_results.insert(0, whole_ids[test_index])
    test_metrics.append(test_metric)
    results.append(final_importance_results)

    torch.save(final_importance_results, 'data/explain/final_importance_results_vocab120_month' + m)

    print(test_metrics, np.mean(test_metrics))

parameters: embsize:16, nhead:1, nhid:32, nlayers:1, dropout:0.5
whole data is done
device:  cuda
----------
Epoch 1/2
----------
epoch_train_loss: 0.08049642046292622 epoch_train_metric: 0.5050934634299812
epoch_val_loss: 0.04875092721525945 epoch_val_metric: 0.6793677014205476
epoch_test_loss: 0.04875092721525945 epoch_test_metric: 0.6793677014205476
updated
----------
Epoch 2/2
----------
epoch_train_loss: 0.046796630483631994 epoch_train_metric: 0.6218608865031598
epoch_val_loss: 0.04630436706265398 epoch_val_metric: 0.6983356397889089
epoch_test_loss: 0.04630436706265398 epoch_test_metric: 0.6983356397889089
updated
[0.6983356397889089] 0.6983356397889089
whole data is done
device:  cuda
----------
Epoch 1/2
----------
epoch_train_loss: 0.04704082406619016 epoch_train_metric: 0.6715232380852993
epoch_val_loss: 0.04735779590612533 epoch_val_metric: 0.700783466856492
epoch_test_loss: 0.04735779590612533 epoch_test_metric: 0.700783466856492
updated
----------
Epoch 2/2
----------
epo

In [15]:


train_batch_size = 2000
test_batch_size = 2000

ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 16 # embedding dimension
nhid = 32 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 1 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
n_class = 20

test_metrics = []
results = []


#criterion to use
criterion = nn.BCEWithLogitsLoss(pos_weight=None).cuda()
# model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, n_class, num_events=500, seq_length=120, dropout=0.5).to(device)

# if torch.cuda.is_available():
#     model = model.cuda()
# if torch.cuda.device_count()>1:
#     model = nn.DataParallel(model)
    
# Training month by month
for i in [12]:
    if i < 10:
        m = '0' + str(i)
    else:
        m = str(i)
    whole_ids, whole_data, whole_labels, whole_mask = ReadData('data/ae_process/np_ae_last120_vocab120_non3digit_month' + m + '.pkl', event_length=500)
    
    #train_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    #val_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    #test_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_2011' + m + '01.csv').to_numpy()
    
#     train_index = np.where(np.in1d(whole_ids, train_ids))[0]
#     val_index = np.where(np.in1d(whole_ids, val_ids))[0]
#     test_index = np.where(np.in1d(whole_ids, test_ids))[0]
#     print('whole data is done')
    
    X_train, y_train, mask_train  = whole_data, whole_labels, whole_mask
    #X_train, y_train, mask_train  = whole_data[train_index], whole_labels[train_index], whole_mask[train_index]
#     X_val, y_val, mask_val = whole_data[val_index], whole_labels[val_index], whole_mask[val_index]
#     X_test, y_test, mask_test = whole_data[test_index], whole_labels[test_index], whole_mask[test_index]

    train_dataset = BuildDataset('', event_length=500, data_list=[X_train, y_train, mask_train], mode='load')
#     val_dataset = BuildDataset('', event_length=500, data_list=[X_val, y_val, mask_val], mode='load')
#     test_dataset = BuildDataset('', event_length=500, data_list=[X_test, y_test, mask_test], mode='load')

    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=12, drop_last=True)
#     val_dataloader = DataLoader(val_dataset, batch_size=train_batch_size, shuffle=False, num_workers=12, drop_last=True)
#     test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=12, drop_last=True)



    epoch=2
    train_metrics, val_metric, test_metric, final_importance_results = TrainingProcess(model, epoch, criterion, [train_dataloader], m)

   # final_importance_results.insert(0, whole_ids[test_index])
    #test_metrics.append(test_metric)
    #results.append(final_importance_results)

    torch.save(final_importance_results, 'data/explain/final_importance_results_vocab120_month' + m)

   # print(test_metrics, np.mean(test_metrics))

parameters: embsize:16, nhead:1, nhid:32, nlayers:1, dropout:0.5
device:  cuda
----------
Epoch 1/2
----------
epoch_train_loss: 0.04575593143917797 epoch_train_metric: 0.6520541037977222
updated
----------
Epoch 2/2
----------
epoch_train_loss: 0.040199166994374785 epoch_train_metric: 0.6932024366433348
updated


In [16]:
epoch=2
train_metrics, val_metric, test_metric, final_importance_results = TrainingProcess(model, epoch, criterion, [train_dataloader], m)

# final_importance_results.insert(0, whole_ids[test_index])
#test_metrics.append(test_metric)
#results.append(final_importance_results)


device:  cuda
----------
Epoch 1/2
----------
epoch_train_loss: 0.040213182440769706 epoch_train_metric: 0.6922007540921401
updated
----------
Epoch 2/2
----------
epoch_train_loss: 0.04014175395066002 epoch_train_metric: 0.6948991662836519
updated


In [37]:
# whole_ids, whole_data, whole_labels, whole_mask, whole_pos = ReadData('data/ae_process/np_ae_last120_non3digit_month01.pkl', event_length=500)
    
# #train_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_20110101.csv').to_numpy()
# #val_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_20110101.csv').to_numpy()
# test_ids = pd.read_csv('data/splits_new/splits_per_month/train_ae_patients_365_20110101.csv').to_numpy()

# #train_index = np.where(np.in1d(whole_ids, train_ids))[0]
# #val_index = np.where(np.in1d(whole_ids, val_ids))[0]
# test_index = np.where(np.in1d(whole_ids, test_ids))[0]
# print('whole data is done')


# #X_train, y_train, mask_train  = whole_data[train_index], whole_labels[train_index], whole_mask[train_index]
# #X_val, y_val, mask_val = whole_data[val_index], whole_labels[val_index], whole_mask[val_index]
# X_test, y_test, mask_test = whole_data[test_index], whole_labels[test_index], whole_mask[test_index]

# #train_dataset = BuildDataset('', event_length=500, data_list=[X_train, y_train, mask_train], mode='load')
# #val_dataset = BuildDataset('', event_length=500, data_list=[X_val, y_val, mask_val], mode='load')
# test_dataset = BuildDataset('', event_length=500, data_list=[X_test, y_test, mask_test], mode='load')

# #train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=6, drop_last=True)
# #val_dataloader = DataLoader(val_dataset, batch_size=train_batch_size, shuffle=False, num_workers=6, drop_last=True)
# test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=6, drop_last=True)
optimizer = optim.AdamW(model.parameters())
epoch_test_loss, epoch_test_metric, importance_results = EopochVal(model, test_dataloader, optimizer, criterion, device="cuda", metric='auc')
        #test_loss.append(epoch_test_loss)
print('epoch_test_loss:',np.mean(epoch_test_loss), 'epoch_test_metric:', np.mean(epoch_test_metric))

epoch_test_loss: 0.04407568524646408 epoch_test_metric: 0.7277205219804261


In [34]:
!aws s3 cp data/ae_pos_vocab_last90_whole_non3 s3://cmsai-mrk-amzn/ae_tranf_submission_2/

upload: data/ae_pos_vocab_last90_whole_non3 to s3://cmsai-mrk-amzn/ae_tranf_submission_2/ae_pos_vocab_last90_whole_non3


In [32]:
!aws s3 cp data/explain/final_importance_results_month11 s3://cmsai-mrk-amzn/xianzeng/explain/

upload: data/explain/final_importance_results_month11 to s3://cmsai-mrk-amzn/xianzeng/explain/final_importance_results_month11


In [20]:
import metrics

In [34]:
labels, scores = np.array(results[-1][1]), np.array(results[-1][4])

In [35]:
labels.shape

(256500, 20)

In [60]:
df = metrics.compute_metrics(labels, scores, 0.2, target_names=None, risk_list=[0.5, 1, 2, 5])

In [61]:
df

,auroc,avgpr,precis,precis 0.5%,precis 1%,precis 2%,precis 5%,recall,recall 0.5%,recall 1%,recall 2%,recall 5%,calibration
0,0.705216,0.056086,0.020033,0.113796,0.113840,0.102534,0.081871,0.999805,1.0,1.0,1.0,1.0,0.371622
1,0.775094,0.078213,0.029508,0.168355,0.148538,0.115595,0.100663,0.874975,1.0,1.0,1.0,1.0,0.285384
2,0.830647,0.140016,0.025907,0.311769,0.240936,0.165887,0.102924,0.855250,1.0,1.0,1.0,1.0,0.229075
3,0.674638,0.044499,0.020187,0.098207,0.090448,0.078752,0.063938,1.000000,1.0,1.0,1.0,1.0,0.405784
4,0.656049,0.031679,0.015552,0.063133,0.064327,0.053801,0.044366,1.000000,1.0,1.0,1.0,1.0,0.442189
5,0.648587,0.034082,0.011806,0.098987,0.078363,0.062963,0.045458,0.999670,1.0,1.0,1.0,1.0,0.394406
6,0.779144,0.060517,0.015906,0.152767,0.123587,0.096101,0.068538,0.865598,1.0,1.0,1.0,1.0,0.298706
7,0.672521,0.028438,0.014051,0.043648,0.042105,0.038596,0.038285,1.000000,1.0,1.0,1.0,1.0,0.400970
8,0.737316,0.055144,0.037138,0.076383,0.075244,0.065107,0.059805,0.937537,1.0,1.0,1.0,1.0,0.311747
9,0.774836,0.029763,0.004474,0.070148,0.054191,0.039376,0.025341,0.923852,1.0,1.0,1.0,1.0,0.272082


In [133]:
df.head()

,auroc,avgpr,precis,precis 0.5%,precis 1%,precis 2%,precis 5%,recall,recall 0.5%,recall 1%,recall 2%,recall 5%,calibration
0,0.678534,0.26095,0.392842,0.431222,0.40704,0.392842,0.392842,0.040343,1.0,1.0,0.778878,0.352699,0.027826
